In [1]:
from astropy.coordinates import SkyCoord, get_moon, EarthLocation, ICRS, GCRS,AltAz
from astropy.time import Time
from astropy.io.fits.hdu.hdulist import HDUList
from datetime import date
from astral import Astral, Location
from astropy.time import Time
import astropy.units as u

In [1]:
import pyfits
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter

In [2]:
## Finding out which file years to download
all_sky=pd.read_csv('objSKY.csv')
all_sky.head()
print('earliest date=', min(all_sky['MJD']), 'or 12-11-2009')
print('latest date=', max(all_sky['MJD']), 'or 05-12-2016')

earliest date= 55176 or 12-11-2009
latest date= 57520 or 05-12-2016


## Data

In [3]:
## To check for reading in the right file
rand_files=('spec-3586-55181-0496.fits','spec-3586-55181-0788.fits','spec-3586-55181-0996.fits',
            'spec-3761-55272-0008.fits','spec-10000-57346-0334.fits','spec-3761-55272-0475.fits',
            'spec-10000-57346-0659.fits','spec-5478-56014-0654.fits','spec-10000-57346-0955.fits',
            'spec-5478-56014-0716.fits','spec-10000-57346-0994.fits')

## To look for patterns
inseq_files=('spec-3663-55176-0010.fits','spec-3663-55176-0012.fits','spec-3663-55176-0020.fits',
             'spec-3663-55176-0024.fits','spec-3663-55176-0036.fits','spec-3663-55176-0038.fits',
             'spec-3663-55176-0048.fits','spec-3663-55176-0052.fits','spec-3663-55176-0056.fits',
             'spec-3663-55176-0068.fits','spec-3663-55176-0075.fits','spec-3663-55176-0078.fits',
             'spec-3663-55176-0090.fits','spec-3663-55176-0094.fits','spec-3663-55176-0096.fits',
             'spec-3663-55176-0108.fits','spec-3663-55176-0112.fits','spec-3663-55176-0114.fits',
             'spec-3663-55176-0128.fits','spec-3663-55176-0134.fits')

solar_files=('g2009.txt','g2010.txt','g2011.txt','g2012.txt','g2013.txt','g2014.txt','g2015.txt','g2016.txt')

## Getting MJDs from fits

In [4]:
#a) first load all files and convert all dates into mjds.
def get_mjd(file):
    da=pyfits.open(file)
    return (da[4].header['MJD'])

In [5]:
for i in rand_files:
    print(get_mjd(i))

55181
55181
55181
55272
57345
55272
57345
56014
57345
56014
57345


## Organizing by date and sum duplicates

In [11]:
def merge_subs(lst_of_lsts):
    res = []
    for row in lst_of_lsts:
        for i, resrow in enumerate(res):
            if row[:3]==resrow[:3]:
                res[i] += row[1:]
                break
        else:
            res.append(row)
    return (res)
## Merges lists, but does not remove first 3 cols

In [12]:
def f2(seq): 
    # order preserving
    checked = []
    for e in seq:
        if e not in checked:
            checked.append(e)
    return checked
## removes duplicates

In [35]:
## Function to organize mill. of sol. disk by date
def organize_sum_dates(txt_file):
    ## We only need year, month, day of month, and mill. of sol. disk
    ## Cols         1-4    5-6      7-8               31-34
    demo=list(chain.from_iterable((x[:4], x[4:6], x[6:10], x[31:34]) for x in open(txt_file).readlines()))
    ## Organizing cols
    str_list=[demo[i*4:i*4+4] for i in range(int(len(demo)/4))]
    ## Converting to float
    float_dates=[]
    for k in range(len(str_list)):
        float_dates.append([float(i) for i in str_list[k]])
    b=([x[:4] for x in float_dates])
    ve=merge_subs(b)
    new=[]
    summed_mills=[]
    no_mills=[]
    merged_lists=[]
    for i in range(len(ve)):
        new.append(f2(ve[i]))
        summed_mills.append(sum(new[i][3:]))
        no_mills.append(new[i][:3])
        merged_lists.append(np.append(no_mills[i],summed_mills[i]).tolist())
    return(merged_lists)

In [40]:
trial=organize_sum_dates(solar_files[2])
trial[:5]

[[2011.0, 1.0, 1.5, 405.0],
 [2011.0, 1.0, 2.5, 375.0],
 [2011.0, 1.0, 3.5, 517.0],
 [2011.0, 1.0, 4.5, 537.0],
 [2011.0, 1.0, 5.5, 381.0]]

In [8]:
#for i in solar_files:
#    print(organize_by_date(i))

## Interpolated function from MJDs to mill of sol. disk

In [18]:
#b) Then set up one massive interpolated function from mjds to mill of sol. disk
## Pulling out cols of interest
demo=list(chain.from_iterable((x[:4], x[4:6], x[6:10], x[31:34]) for x in open('g2010.txt').readlines()))
## Organizing cols
str_list=[demo[i*4:i*4+4] for i in range(int(len(demo)/4))]
## Converting to float
float_dates=[]
for k in range(len(str_list)):
    float_dates.append([float(i) for i in str_list[k]])
b=([x[:4] for x in float_dates])
ve=merge_subs(b)
new=[]
summed_mills=[]
no_mills=[]
merged_lists=[]
for i in range(len(ve)):
    new.append(f2(ve[i]))
    summed_mills.append(sum(new[i][3:]))
    no_mills.append(new[i][:3])
    merged_lists.append(np.append(no_mills[i],summed_mills[i]).tolist())

In [19]:
merged_lists

[[2010.0, 1.0, 1.5, 180.0],
 [2010.0, 1.0, 2.5, 242.0],
 [2010.0, 1.0, 3.5, 210.0],
 [2010.0, 1.0, 4.5, 89.0],
 [2010.0, 1.0, 5.5, 0.0],
 [2010.0, 1.0, 7.5, 67.0],
 [2010.0, 1.0, 8.5, 44.0],
 [2010.0, 1.0, 9.5, 94.0],
 [2010.0, 1.0, 10.5, 204.0],
 [2010.0, 1.0, 11.5, 493.0],
 [2010.0, 1.0, 12.5, 505.0],
 [2010.0, 1.0, 13.5, 628.0],
 [2010.0, 1.0, 14.5, 420.0],
 [2010.0, 1.0, 15.5, 432.0],
 [2010.0, 1.0, 16.5, 251.0],
 [2010.0, 1.0, 17.5, 62.0],
 [2010.0, 1.0, 18.5, 21.0],
 [2010.0, 1.0, 20.5, 28.0],
 [2010.0, 1.0, 21.5, 181.0],
 [2010.0, 1.0, 22.5, 309.0],
 [2010.0, 1.0, 23.5, 544.0],
 [2010.0, 1.0, 24.5, 452.0],
 [2010.0, 1.0, 25.5, 116.0],
 [2010.0, 1.0, 26.5, 121.0],
 [2010.0, 1.0, 27.5, 72.0],
 [2010.0, 1.0, 28.5, 48.0],
 [2010.0, 1.0, 29.5, 28.0],
 [2010.0, 1.0, 30.5, 64.0],
 [2010.0, 1.0, 31.5, 93.0],
 [2010.0, 2.0, 1.5, 117.0],
 [2010.0, 2.0, 2.5, 85.0],
 [2010.0, 2.0, 3.5, 81.0],
 [2010.0, 2.0, 4.5, 28.0],
 [2010.0, 2.0, 5.5, 35.0],
 [2010.0, 2.0, 6.5, 497.0],
 [2010.0, 2.0, 7.

## Finding the duplicates, averaging mill of sol. disk, remove duplicate rows

In [13]:
ve=merge_subs(b)

In [14]:
new=[]
summed_mills=[]
no_mills=[]
merged_lists=[]
for i in range(len(ve)):
    new.append(f2(ve[i]))
    summed_mills.append(sum(new[i][3:]))
    no_mills.append(new[i][:3])
    merged_lists.append(np.append(no_mills[i],summed_mills[i]).tolist())

In [15]:
merged_lists

[[2011.0, 1.0, 1.5, 405.0],
 [2011.0, 1.0, 2.5, 375.0],
 [2011.0, 1.0, 3.5, 517.0],
 [2011.0, 1.0, 4.5, 537.0],
 [2011.0, 1.0, 5.5, 381.0],
 [2011.0, 1.0, 6.5, 365.0],
 [2011.0, 1.0, 7.5, 244.0],
 [2011.0, 1.0, 8.5, 233.0],
 [2011.0, 1.0, 9.5, 153.0],
 [2011.0, 1.0, 10.5, 112.0],
 [2011.0, 1.0, 11.5, 59.0],
 [2011.0, 1.0, 12.5, 16.0],
 [2011.0, 1.0, 13.5, 17.0],
 [2011.0, 1.0, 14.5, 0.0],
 [2011.0, 1.0, 15.5, 30.0],
 [2011.0, 1.0, 16.5, 81.0],
 [2011.0, 1.0, 17.5, 171.0],
 [2011.0, 1.0, 18.5, 255.0],
 [2011.0, 1.0, 19.5, 217.0],
 [2011.0, 1.0, 20.5, 305.0],
 [2011.0, 1.0, 21.5, 283.0],
 [2011.0, 1.0, 22.5, 403.0],
 [2011.0, 1.0, 23.5, 360.0],
 [2011.0, 1.0, 24.5, 329.0],
 [2011.0, 1.0, 25.5, 176.0],
 [2011.0, 1.0, 26.5, 141.0],
 [2011.0, 1.0, 28.5, 27.0],
 [2011.0, 1.0, 29.5, 27.0],
 [2011.0, 1.0, 30.5, 77.0],
 [2011.0, 1.0, 31.5, 124.0],
 [2011.0, 2.0, 1.5, 151.0],
 [2011.0, 2.0, 2.5, 155.0],
 [2011.0, 2.0, 3.5, 37.0],
 [2011.0, 2.0, 4.5, 161.0],
 [2011.0, 2.0, 5.5, 43.0],
 [2011.0, 2